In [2]:
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from torchvision import models
from torch import Tensor, nn

transfer_model = models.resnet50(pretrained=True)


In [14]:
for name, param in transfer_model.named_parameters():
  param.requires_grad = False

In [15]:
transfer_model.fc = nn.Sequential(nn.Linear(transfer_model.fc.in_features,500),
    nn.ReLU(),
    nn.Dropout(), nn.Linear(500,2))

In [17]:
import math
def find_lr(model, loss_fn, optimizer, init_value=1e-8, final_value=10.0):
  number_in_epoch = len(train_loader) - 1
  update_step = (final_value / init_value) ** (1 / number_in_epoch)
  lr = init_value
  optimizer.param_groups[0]["lr"] = lr
  best_loss = 0.0
  batch_num = 0
  losses = []
  log_lrs = []
  for data in train_loader:
    batch_num += 1
    inputs, labels = data
    inputs, labels = inputs, labels
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = loss_fn(outputs, labels)
    # Crash out if loss explodes
    if batch_num > 1 and loss > 4 * best_loss: return log_lrs[10:-5], losses[10:-5]
    # Record the best loss
    if loss < best_loss or batch_num == 1: best_loss = loss
    # Store the values
    losses.append(loss)
    log_lrs.append(math.log10(lr))
    # Do the backward pass and optimize
    loss.backward()
    optimizer.step()
    # Update the lr for the next step and store
    lr *= update_step
    optimizer.param_groups[0]["lr"] = lr
  return log_lrs[10:-5], losses[10:-5]
